<a href="https://colab.research.google.com/github/abhikr1302/Data_Extraction/blob/main/Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Blackcoffer Test Assignment**

# **Data Extraction and NLP**


## **Objective**
The objective of this assignment is to extract textual data articles from the given URL and perform text analysis to compute variables that are explained below. 


## **Importing The Libraries**

In [ ]:
!pip install fake_useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 5.5 MB/s 


In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import requests.exceptions
from fake_useragent import UserAgent
from datetime import datetime
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
import glob
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
 #importing the drive
 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Data Study** 

In [ ]:
#Loading File

df=pd.read_excel('/content/drive/MyDrive/20211030 Test Assignment/Input.xlsx')

In [ ]:
df.head()

,URL_ID,URL
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...
1,38.0,https://insights.blackcoffer.com/what-if-the-c...
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...
3,40.0,https://insights.blackcoffer.com/will-machine-...
4,41.0,https://insights.blackcoffer.com/will-ai-repla...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   URL_ID  114 non-null    float64
 1   URL     114 non-null    object 
dtypes: float64(1), object(1)
memory usage: 1.9+ KB


In [ ]:
df['URL'][0]

'https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/'

## **Data Manipulation**

In [ ]:
# Function to find if there is any error in URL 
urls = df['URL'].tolist()
status = []

## initializing the UserAgent object
user_agent = UserAgent()

for url in urls:
    try:
        r = requests.get(url,headers={"user-agent": user_agent.chrome})
        r.raise_for_status()
    except (requests.exceptions.ConnectionError, requests.exceptions.Timeout):
        status.append("Down")
    except requests.exceptions.HTTPError:
        status.append("HTTP error")
    else:
        status.append("All good")
df['status'] = status

In [ ]:
df.loc[df['status']=='HTTP error']

,URL_ID,URL,status
7,44.0,https://insights.blackcoffer.com/how-neural-ne...,HTTP error
20,57.0,https://insights.blackcoffer.com/covid-19-envi...,HTTP error
107,144.0,https://insights.blackcoffer.com/ensuring-grow...,HTTP error


We find that three URLs are not working.

In [ ]:
# removing rows that have HTTP error
df = df.loc[df['status']=='All good']
df

,URL_ID,URL,status
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,All good
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,All good
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,All good
3,40.0,https://insights.blackcoffer.com/will-machine-...,All good
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,All good
...,...,...,...
109,146.0,https://insights.blackcoffer.com/blockchain-fo...,All good
110,147.0,https://insights.blackcoffer.com/the-future-of...,All good
111,148.0,https://insights.blackcoffer.com/big-data-anal...,All good
112,149.0,https://insights.blackcoffer.com/business-anal...,All good


In [ ]:
# Function to find date of the article when it is uploaded
urls = df['URL'].tolist()
Date = []

## initializing the UserAgent object
user_agent = UserAgent()

#scrape elements
for url in urls:
  response = requests.get(url, headers={"user-agent": user_agent.chrome})
  soup = BeautifulSoup(response.content, "html.parser")

  ## Then parse the HTML, extract title and article
  date = soup.find(attrs={'class':'entry-date updated td-module-date'}).text
  Date.append(date)

df['date'] = Date

In [ ]:
df

,URL_ID,URL,status,date
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,All good,"June 26, 2021"
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,All good,"June 25, 2021"
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,All good,"June 25, 2021"
3,40.0,https://insights.blackcoffer.com/will-machine-...,All good,"June 25, 2021"
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,All good,"June 25, 2021"
...,...,...,...,...
109,146.0,https://insights.blackcoffer.com/blockchain-fo...,All good,"September 7, 2019"
110,147.0,https://insights.blackcoffer.com/the-future-of...,All good,"August 31, 2019"
111,148.0,https://insights.blackcoffer.com/big-data-anal...,All good,"July 19, 2019"
112,149.0,https://insights.blackcoffer.com/business-anal...,All good,"July 19, 2019"


## **Data Extraction**

### **Function to extract Title and Text from Article monthwise**

In [ ]:
# Converting data into datetime format
df['date']= pd.to_datetime(df['date'])
df['date']

0     2021-06-26
1     2021-06-25
2     2021-06-25
3     2021-06-25
4     2021-06-25
         ...    
109   2019-09-07
110   2019-08-31
111   2019-07-19
112   2019-07-19
113   2019-07-19
Name: date, Length: 111, dtype: datetime64[ns]

In [ ]:
# Using pandas.DatetimeIndex() to extract month and year
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month

### **January**

In [ ]:
jan_df = df.loc[df['month']==1]

In [ ]:
#Function to extract Title and text from the article published in January 
## create an array with URLs
urls = jan_df['URL']

## initializing the UserAgent object
user_agent = UserAgent()

# defining empty list for title and text
title_all = []
text_all = []

#scrape elements
for url in urls:
  response = requests.get(url, headers={"user-agent": user_agent.chrome})
  soup = BeautifulSoup(response.content, "html.parser")

  ## Then parse the HTML, extract title and article
  jan_title = soup.find(attrs={'class':'td-post-title'}).text.replace('\n','')
  jan_title = (re.findall('Think(.*)January', jan_title))
  jan_text = soup.find(attrs={'class':'td-post-content'}).text.replace('\n','')
  
  if jan_title == []:
    print("Title could not be found")
    continue
  else:
    print(f"Title of the article is : {jan_title} \nText of the article is : {jan_text}")
# get the domain as the file name
  domain = " ".join(jan_title) 
  description = jan_text
  filename = response.url.replace("/",'') + '.txt'
  with open("/content/drive/MyDrive/20211030 Test Assignment/output/" + filename, 'w') as f:
      f.write(domain +'\n' + description)

# appending all the title and text in a list
  domain = " ".join(jan_title) 
  description = jan_text
  title_all.append(domain)
  text_all.append(description)

Title of the article is : [' What patients like and dislike about telemedicine?'] 
Text of the article is : In today’s world, telemedicine technology is one of those technologies which has brought about a change. Compared to the early days there have been remarkable differences in the methods of consultation with a doctor. In the years that have passed by, consultation for a disease with a doctor was quite hectic. It involved waiting, traveling, etc. But with the advent of telemedicine opportunities, this has completely changed.It is a rural area that has been completely blessed with the invention of telemedicine. Today a considerable amount of people are able to consult doctors remotely. Not just doctors, but specialists in various fields of medicine. This has been of great importance as far as rural people are concerned. There are a lot of telemedicine tools that have been found. There are a lot of areas like ophthalmology, oncology, dermatology, etc where the facility of telemedicin

In [ ]:
# adding title and text to the dataframe
jan_df.insert(2,'Title', title_all)
jan_df.insert(3,'Text', text_all)
jan_df

,URL_ID,URL,Title,Text,status,date,year,month
28,65.0,https://insights.blackcoffer.com/telemedicine-...,What patients like and dislike about telemedi...,"In today’s world, telemedicine technology is o...",All good,2021-01-13,2021,1
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,How we forecast future technologies?,TECHNOLOGY is playing a dominant role in human...,All good,2021-01-13,2021,1


### **February**

In [ ]:
feb_df = df.loc[df['month']==2]

In [ ]:
#Function to extract Title and text from the article published in February
## create an array with URLs
urls = feb_df['URL']

## initializing the UserAgent object
user_agent = UserAgent()

# defining empty list for title and text
title_all = []
text_all = []

#scrape elements
for url in urls:
  response = requests.get(url, headers={"user-agent": user_agent.chrome})
  soup = BeautifulSoup(response.content, "html.parser")

  ## Then parse the HTML, extract title and article
  feb_title = soup.find(attrs={'class':'td-post-title'}).text.replace('\n','')
  feb_title = (re.findall('Think(.*)February', feb_title))
  feb_text = soup.find(attrs={'class':'td-post-content'}).text.replace('\n','')

  if feb_title == []:
    print("Title could not be found")
    continue
  else:
    print(f"Title of the article is : {feb_title} \nText of the article is : {feb_text}")
# get the domain as the file name
  domain = " ".join(feb_title) 
  description = feb_text
  filename = response.url.replace("/",'') + '.txt'
  with open("/content/drive/MyDrive/20211030 Test Assignment/output/" + filename, 'w') as f:
      f.write(domain +'\n' + description)

  # appending all the title and text in a list
  domain = " ".join(feb_title) 
  description = feb_text
  title_all.append(domain)
  text_all.append(description)

Title of the article is : [' What is the future of mobile apps?'] 
Tech Giants such as Apple and Google have launched their own augmented reality kits — AR Kit and AR Core resp to help app developers to create high-quality mobile apps. As for VR, the latest buzzword associated with it is v-commerce. One of the examples is Alibaba, which in November of 2016 introduced VR shopping to its customers across China.Blackcoffer Insights 24: Nikhil Singh, St.peter's college agra
Title of the article is : [' Impact of AI in health and medicine'] 
Text of the article is : AI allows those in training to go through naturalistic simulations in a way that simple computer-driven algorithms cannot. The advent of natural speech and the ability of an AI  computer to draw instantly on a large database of scenarios means the response to questions, decisions, or advice from a trainee can challenge in a way that humans cannot in health and medicineHealth monitoring:Wearable health trackers-like those from Fi

In [ ]:
# adding title and text to the dataframe
feb_df.insert(2,'Title', title_all)
feb_df.insert(3,'Text', text_all)
feb_df

,URL_ID,URL,Title,Text,status,date,year,month
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,What is the future of mobile apps?,From what I have learned on the mobile apps de...,All good,2021-02-12,2021,2
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,Impact of AI in health and medicine,AI allows those in training to go through natu...,All good,2021-02-11,2021,2


### **March**

In [ ]:
mar_df = df.loc[df['month']==3]

In [ ]:
#Function to extract Title and text from the article published in March
## create an array with URLs
urls = mar_df['URL']

## initializing the UserAgent object
user_agent = UserAgent()

# defining empty list for title and text
title_all = []
text_all = []

#scrape elements
for url in urls:
  response = requests.get(url, headers={"user-agent": user_agent.chrome})
  soup = BeautifulSoup(response.content, "html.parser")

  ## Then parse the HTML, extract title and article
  mar_title = soup.find(attrs={'class':'td-post-title'}).text.replace('\n','')
  mar_title = (re.findall('Think(.*)March', mar_title))
  mar_text = soup.find(attrs={'class':'td-post-content'}).text.replace('\n','')

  if mar_title == []:
    print("Title could not be found")
    continue
  else:
    print(f"Title of the article is : {mar_title} \nText of the article is : {mar_text}")

  # get the domain as the file name
  domain = " ".join(mar_title) 
  description = mar_text
  filename = response.url.replace("/",'') + '.txt'
  with open("/content/drive/MyDrive/20211030 Test Assignment/output/" + filename, 'w') as f:
      f.write(domain +'\n' + description)

  # appending all the title and text in a list
  domain = " ".join(mar_title) 
  description = mar_text
  title_all.append(domain)
  text_all.append(description)

Title of the article is : [' What is the difference between Artificial Intelligence, Machine Learning, Statistics, and Data Mining?'] 
Text of the article is : “Data is the new oil” has become the most important trendline of the 21st century. The reason for this is the advancements in the fields related to data analysis. The field of AI Machine Learning Statistics and Data Mining all deal with data and are developing at such a staggering pace, that these fields have become the most popular buzzwords these days.Buzzwords are originated through technical terms but often the underlying essence is ignored through fashionable use and mainly used to impress. This is the main reason for the misconception amongst people. AI, ML, Stats, Data Mining, and many other fields related to the analysis of data are most often mistook for one and the same thing, thus all these words are often used interchangeably to convey one and the same thing. But this is not true at all!The only similarity between th

**Title of one Article was not found. Lets find it out now**

In [ ]:
mar_df

,URL_ID,URL,status,date,year,month
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3
24,61.0,https://insights.blackcoffer.com/how-python-be...,All good,2021-03-04,2021,3
25,62.0,https://insights.blackcoffer.com/how-google-fi...,All good,2021-03-04,2021,3
97,134.0,https://insights.blackcoffer.com/coronavirus-d...,All good,2020-03-30,2020,3
98,135.0,https://insights.blackcoffer.com/should-people...,All good,2020-03-30,2020,3
99,136.0,https://insights.blackcoffer.com/why-is-there-...,All good,2020-03-30,2020,3
100,137.0,https://insights.blackcoffer.com/what-do-you-t...,All good,2020-03-30,2020,3
101,138.0,https://insights.blackcoffer.com/coronavirus-t...,All good,2020-03-30,2020,3


**We can see that Title of URL in index 98 can't be extracted. Lets find it out now.**

In [ ]:
# Finding the title of article which was not found earlier
urls = 'https://insights.blackcoffer.com/should-people-wear-fabric-gloves-seeking-evidence-regarding-the-differential-transfer-of-covid-19-or-coronaviruses-generally-between-surfaces/'

## initializing the UserAgent object
user_agent = UserAgent()

#scrape elements
response = requests.get(urls, headers={"user-agent": user_agent.chrome})
soup = BeautifulSoup(response.content, "html.parser")

## Then parse the HTML, extract title and article
mar_title = soup.find(attrs={'class':'td-post-title'}).text.replace('\n','')
mar_title = (re.findall('Sciences(.*)March', mar_title))
mar_text = soup.find(attrs={'class':'td-post-content'}).text.replace('\n','')

print(f"Title of the article is : {mar_title} \nText of the article is : {mar_text}")

# get the domain as the file name
domain = " ".join(mar_title) 
description = mar_text
filename = response.url.replace("/",'') + '.txt'
with open("/content/drive/MyDrive/20211030 Test Assignment/output/" + filename, 'w') as f:
  f.write(domain +'\n' + description)

# appending all the title and text in a list
domain = " ".join(mar_title) 
description = mar_text
title_all.append(domain)
text_all.append(description)

Title of the article is : [' Should people wear fabric gloves? Seeking evidence regarding the differential transfer of COVID-19 (or coronaviruses generally) between surfaces?'] 
Text of the article is : Is there any evidence about whether fabric or bare hands would spread covid-19 more from one surface to another? (e.g. opening a door using a fabric glove). Assuming the fabric doesn’t touch your face, like bare hands easily do, even with efforts to remember not to people struggle to remember. I’ve been thinking of prompts to prevent face touching, such as wearing fabric gloves when out and about, but I don’t want to promote this unless the fabric is at-least-no-worse than hands for transferring the virus between surfaces.I’m talking about use in the community for instance when people have to travel or go to food shops, not in a healthcare context. The gloves should be changed between different settings (e.g. apartment block, bus, shop – switch to a new pair of gloves for each). The glo

In [ ]:
title_all

[' What is the difference between Artificial Intelligence, Machine Learning, Statistics, and Data Mining?',
 ' How Python became the first choice for Data Science.',
 ' How Google fit measure heart and respiratory rates using a phone camera?',
 ' Coronavirus Disease (COVID-19) Effect: The Impact and Role of Mass Media During the Pandemic',
 ' Why is there a severe immunological and inflammatory explosion in those affected by sarms covid-19?',
 ' What do you think is the lesson (or lessons) to be learned with Covid 19?',
 ' Coronavirus, the unexpected challenge for the European union',
 ' Should people wear fabric gloves? Seeking evidence regarding the differential transfer of COVID-19 (or coronaviruses generally) between surfaces?']

In [ ]:
# rearranging dataframe 
mar_df_re = mar_df.reindex([23,24,25,97,99,100,101,98])
mar_df_re

,URL_ID,URL,status,date,year,month
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3
24,61.0,https://insights.blackcoffer.com/how-python-be...,All good,2021-03-04,2021,3
25,62.0,https://insights.blackcoffer.com/how-google-fi...,All good,2021-03-04,2021,3
97,134.0,https://insights.blackcoffer.com/coronavirus-d...,All good,2020-03-30,2020,3
99,136.0,https://insights.blackcoffer.com/why-is-there-...,All good,2020-03-30,2020,3
100,137.0,https://insights.blackcoffer.com/what-do-you-t...,All good,2020-03-30,2020,3
101,138.0,https://insights.blackcoffer.com/coronavirus-t...,All good,2020-03-30,2020,3
98,135.0,https://insights.blackcoffer.com/should-people...,All good,2020-03-30,2020,3


In [ ]:
# adding title and text to the dataframe
mar_df_re.insert(2,'Title', title_all)
mar_df_re.insert(3,'Text', text_all)
mar_df_re

,URL_ID,URL,Title,Text,status,date,year,month
23,60.0,https://insights.blackcoffer.com/difference-be...,What is the difference between Artificial Int...,“Data is the new oil” has become the most impo...,All good,2021-03-09,2021,3
24,61.0,https://insights.blackcoffer.com/how-python-be...,How Python became the first choice for Data S...,Data Science is gaining popularity exponential...,All good,2021-03-04,2021,3
25,62.0,https://insights.blackcoffer.com/how-google-fi...,How Google fit measure heart and respiratory ...,It was in March that Tech giant Google came up...,All good,2021-03-04,2021,3
97,134.0,https://insights.blackcoffer.com/coronavirus-d...,Coronavirus Disease (COVID-19) Effect: The Im...,The outbreak of coronavirus disease 2019 (COVI...,All good,2020-03-30,2020,3
99,136.0,https://insights.blackcoffer.com/why-is-there-...,Why is there a severe immunological and infla...,Why is there a severe immunological and inflam...,All good,2020-03-30,2020,3
100,137.0,https://insights.blackcoffer.com/what-do-you-t...,What do you think is the lesson (or lessons) ...,"What we can learn, is to have frequent updates...",All good,2020-03-30,2020,3
101,138.0,https://insights.blackcoffer.com/coronavirus-t...,"Coronavirus, the unexpected challenge for the...","Since the creation of the EU, we have been thr...",All good,2020-03-30,2020,3
98,135.0,https://insights.blackcoffer.com/should-people...,Should people wear fabric gloves? Seeking evi...,Is there any evidence about whether fabric or ...,All good,2020-03-30,2020,3


### **April**

In [ ]:
apr_df = df.loc[df['month']==4]

In [ ]:
#Function to extract Title and text from the article published in April
## create an array with URLs
urls = apr_df['URL']

## initializing the UserAgent object
user_agent = UserAgent()

# defining empty list for title and text
title_all = []
text_all = []

#scrape elements
for url in urls:
  response = requests.get(url, headers={"user-agent": user_agent.chrome})
  soup = BeautifulSoup(response.content, "html.parser")

  ## Then parse the HTML, extract title and article
  apr_title = soup.find(attrs={'class':'td-post-title'}).text.replace('\n','')
  apr_title = (re.findall('Think(.*)April', apr_title))
  apr_text = soup.find(attrs={'class':'td-post-content'}).text.replace('\n','')
  if apr_title == []:
    print("Title could not be found")
    continue
  else:
    print(f"Title of the article is : {apr_title} \nText of the article is : {apr_text}")
# get the domain as the file name
  domain = " ".join(apr_title) 
  description = apr_text
  filename = response.url.replace("/",'') + '.txt'
  with open("/content/drive/MyDrive/20211030 Test Assignment/output/" + filename, 'w') as f:
      f.write(domain +'\n' + description)
  # appending all the title and text in a list
  domain = " ".join(apr_title) 
  description = apr_text
  title_all.append(domain)
  text_all.append(description)

Title of the article is : [' Environmental impact of the COVID-19 pandemic – Lesson for the Future'] 
Text of the article is : The Covid- 19 pandemics forced factories to shut down, flights getting canceled and a massive decrease in the global economy, with a significant decrease in Green House Gases (GHG) in many developed and developing countries.The SARS- CoV2 came into the spotlight in December 2019 and has impacted most of the countries till then. Nearly 131 million peoples were infected worldwide and resulting in deaths of around 2.9 Million according to World Health Organisation (WHO). Most of the countries dealt with the new virus by imposing strict lockdowns and social distancing to control the spread of the virus. These policies caused adverse effects worldwide. One of the most important impacts of the Covid-19 Pandemic is on the environment.There have been few positive impacts on the environment due to lockdown like, air pollution has decreased dramatically, as people were a

In [ ]:
apr_df

,URL_ID,URL,status,date,year,month
21,58.0,https://insights.blackcoffer.com/environmental...,All good,2021-04-30,2021,4
22,59.0,https://insights.blackcoffer.com/how-data-anal...,All good,2021-04-30,2021,4
71,108.0,https://insights.blackcoffer.com/how-will-covi...,All good,2020-04-29,2020,4
72,109.0,https://insights.blackcoffer.com/lessons-from-...,All good,2020-04-29,2020,4
73,110.0,https://insights.blackcoffer.com/covid-19-how-...,All good,2020-04-29,2020,4
74,111.0,https://insights.blackcoffer.com/coronavirus-i...,All good,2020-04-30,2020,4
75,112.0,https://insights.blackcoffer.com/how-will-covi...,All good,2020-04-30,2020,4
76,113.0,https://insights.blackcoffer.com/coronavirus-i...,All good,2020-04-30,2020,4
77,114.0,https://insights.blackcoffer.com/estimating-th...,All good,2020-04-30,2020,4
78,115.0,https://insights.blackcoffer.com/covid-19-how-...,All good,2020-04-30,2020,4


**We can see that Title of URL in index 102,103,105 can't be extracted. Lets find it out now**

In [ ]:
# Left over articles
urls = 'https://insights.blackcoffer.com/industrial-revolution-4-0-pros-and-cons/'

## initializing the UserAgent object
user_agent = UserAgent()

#scrape elements
response = requests.get(urls, headers={"user-agent": user_agent.chrome})
soup = BeautifulSoup(response.content, "html.parser")

## Then parse the HTML, extract title and article
apr_title = soup.find(attrs={'class':'td-post-title'}).text.replace('\n','')
apr_title = (re.findall('Technology(.*)April', apr_title))
apr_text = soup.find(attrs={'class':'td-post-content'}).text.replace('\n','')

print(f"Title of the article is : {apr_title} \nText of the article is : {apr_text}")
# get the domain as the file name
domain = " ".join(apr_title) 
description = apr_text
filename = response.url.replace("/",'') + '.txt'
with open("/content/drive/MyDrive/20211030 Test Assignment/output/" + filename, 'w') as f:
  f.write(domain +'\n' + description)

# appending all the title and text in a list
domain = " ".join(apr_title) 
description = apr_text
title_all.append(domain)
text_all.append(description)

Title of the article is : [' INDUSTRIAL REVOLUTION 4.0 – PROS AND CONS'] 
Text of the article is : Before jumping to the pros and cons of industrial revolution 4.0, it is important to understand industrial revolution 4.0. Industrial Revolution 4.0: Industrial revolution 4.0 is the term that is used to describe the transition from an electronic-based industry to the one that is dominated by the fusion of new and emerging fields and ideas like Artificial Intelligence, robotics, and green energy, etc. ‘Industry 4.0’, is the era of digitalization. The term originated from Hanover Messe 2011 and was incorporated in German high-tech strategy. Much is attributed to Klaus Schwab, a German engineer who founded the World Economic Forum. Two years later, the Industry 4.0 Platform (central network for digital transformation) was set up to make it work. It is characterized by 4 physical and digital trends: 1 Autonomous motor vehicles (cars, trucks, drones, aircraft, water vessels); 2 3D printing (m

In [ ]:
# Left over articles
urls = 'https://insights.blackcoffer.com/impact-of-covid-19-coronavirus-on-the-indian-economy/'

## initializing the UserAgent object
user_agent = UserAgent()

#scrape elements
response = requests.get(urls, headers={"user-agent": user_agent.chrome})
soup = BeautifulSoup(response.content, "html.parser")

## Then parse the HTML, extract title and article
apr_title = soup.find(attrs={'class':'td-post-title'}).text.replace('\n','')
apr_title = (re.findall('Pharmaceutical(.*)April', apr_title))
apr_text = soup.find(attrs={'class':'td-post-content'}).text.replace('\n','')

print(f"Title of the article is : {apr_title} \nText of the article is : {apr_text}")
# get the domain as the file name
domain = " ".join(apr_title) 
description = apr_text
filename = response.url.replace("/",'') + '.txt'
with open("/content/drive/MyDrive/20211030 Test Assignment/output/" + filename, 'w') as f:
  f.write(domain +'\n' + description)

# appending all the title and text in a list
domain = " ".join(apr_title) 
description = apr_text
title_all.append(domain)
text_all.append(description)

Title of the article is : [' COVID-19 Vs Indian Economy'] 
Text of the article is : India is on the verge of an unprecedented economic as well as humanitarian disaster from COVID-19. India is facing economic pressure due to the coronavirus outbreak. This outbreak has resulted in a steep slowdown in the Indian economy, whereby India’s GDP has reduced by 4.5% as seen by current statistics. With COVID-19 being into the picture the Indian economy is going through a severe slowdown and on top of that multiple such problems hitting the world of work from many such directions. Companies and industries are finding it difficult to sustain in such an environment of the financial crisis. Industries are being urged to take up certain tough decisions like layoff, retrenchment, compulsive leave without pay and many alternative cost-effective methods to sustain the present economic crisis like cutting down the salaries, handing over the pink slip to employees and opting best possible. The sudden outb

In [ ]:
# Left over articles
urls = 'https://insights.blackcoffer.com/impact-of-covid-19coronavirus-on-the-indian-economy/'

## initializing the UserAgent object
user_agent = UserAgent()

#scrape elements
response = requests.get(urls, headers={"user-agent": user_agent.chrome})
soup = BeautifulSoup(response.content, "html.parser")

## Then parse the HTML, extract title and article
apr_title = soup.find(attrs={'class':'td-post-title'}).text.replace('\n','')
apr_title = (re.findall('Healthcare(.*)April', apr_title))
apr_text = soup.find(attrs={'class':'td-post-content'}).text.replace('\n','')

print(f"Title of the article is : {apr_title} \nText of the article is : {apr_text}")
# get the domain as the file name
domain = " ".join(apr_title) 
description = apr_text
filename = response.url.replace("/",'') + '.txt'
with open("/content/drive/MyDrive/20211030 Test Assignment/output/" + filename, 'w') as f:
  f.write(domain +'\n' + description)

# appending all the title and text in a list
domain = " ".join(apr_title) 
description = apr_text
title_all.append(domain)
text_all.append(description)

Title of the article is : [' Impact of Indian Economy due to COVID-19'] 
Text of the article is : COVID-19: INTRODUCTIONThe Novel Coronavirus (COVID-19),Initially emerged in China at the end of  2019 and by the end of January2020, The World Health Organization declared this pandemic as a Public HealthEmergency of International Concern. This viruscauses respiratory illness (like the flu) with symptoms such as a cough, fever,and in more severe cases, difficulty in breathing or may cause lung disease.Senior Citizens or people under some medical treatment are more likely toacclimate to this virus.The impact of thisoutbreak is rapidly increasing at a higher rate than anything else in theeconomies of various countries of the world. This outbreak is considered asituation like recession or slowdown of the overall economy after the greatrecession in the year 2008. India is thefifth-largest economy in the world with a population of 133.92 Crores. India isstill in its developing phase and its une

In [ ]:
# rearranging dataframe 
apr_df_re = apr_df.reindex([21,22,71,72,73,74,75,76,77,78,79,83,84,85,86,87,88,89,90,91,92,93,94,104,106,102,103,105])
apr_df_re

,URL_ID,URL,status,date,year,month
21,58.0,https://insights.blackcoffer.com/environmental...,All good,2021-04-30,2021,4
22,59.0,https://insights.blackcoffer.com/how-data-anal...,All good,2021-04-30,2021,4
71,108.0,https://insights.blackcoffer.com/how-will-covi...,All good,2020-04-29,2020,4
72,109.0,https://insights.blackcoffer.com/lessons-from-...,All good,2020-04-29,2020,4
73,110.0,https://insights.blackcoffer.com/covid-19-how-...,All good,2020-04-29,2020,4
74,111.0,https://insights.blackcoffer.com/coronavirus-i...,All good,2020-04-30,2020,4
75,112.0,https://insights.blackcoffer.com/how-will-covi...,All good,2020-04-30,2020,4
76,113.0,https://insights.blackcoffer.com/coronavirus-i...,All good,2020-04-30,2020,4
77,114.0,https://insights.blackcoffer.com/estimating-th...,All good,2020-04-30,2020,4
78,115.0,https://insights.blackcoffer.com/covid-19-how-...,All good,2020-04-30,2020,4


In [ ]:
# adding title and text to the dataframe
apr_df_re.insert(2,'Title', title_all)
apr_df_re.insert(3,'Text', text_all)
apr_df_re

,URL_ID,URL,Title,Text,status,date,year,month
21,58.0,https://insights.blackcoffer.com/environmental...,Environmental impact of the COVID-19 pandemic...,The Covid- 19 pandemics forced factories to sh...,All good,2021-04-30,2021,4
22,59.0,https://insights.blackcoffer.com/how-data-anal...,How Data Analytics and AI are used to halt th...,Even though COVID-19 has not yet halted and we...,All good,2021-04-30,2021,4
71,108.0,https://insights.blackcoffer.com/how-will-covi...,How will COVID-19 affect the world of work?,As business close to help prevent transmission...,All good,2020-04-29,2020,4
72,109.0,https://insights.blackcoffer.com/lessons-from-...,Lessons from the past: Some key learnings rel...,"So, not beginning with once upon a time becaus...",All good,2020-04-29,2020,4
73,110.0,https://insights.blackcoffer.com/covid-19-how-...,COVID-19: How have countries been responding?,The COVID-19 pandemic is impacting communities...,All good,2020-04-29,2020,4
74,111.0,https://insights.blackcoffer.com/coronavirus-i...,How Coronavirus Impact on the Hospitality Ind...,While Coronavirus has affected the life of eve...,All good,2020-04-30,2020,4
75,112.0,https://insights.blackcoffer.com/how-will-covi...,How will COVID-19 affect the world of work?,A respiratory illness that started from patien...,All good,2020-04-30,2020,4
76,113.0,https://insights.blackcoffer.com/coronavirus-i...,Coronavirus: Effect on the Hospitality Industry,The hospitality sector in India has witnessed ...,All good,2020-04-30,2020,4
77,114.0,https://insights.blackcoffer.com/estimating-th...,Estimating the impact of COVID-19 on the worl...,"“Stay at Home”, “Crisis”, “Pandemic”, “COVID-1...",All good,2020-04-30,2020,4
78,115.0,https://insights.blackcoffer.com/covid-19-how-...,COVID-19: How have countries been responding?,“If anything kills over 10 million people over...,All good,2020-04-30,2020,4


### **May**

In [ ]:
may_df = df.loc[df['month']==5]

In [ ]:
#Function to extract Title and text from the article published in May
## create an array with URLs
urls = may_df['URL']

## initializing the UserAgent object
user_agent = UserAgent()

# defining empty list for title and text
title_all = []
text_all = []

#scrape elements
for url in urls:
  response = requests.get(url, headers={"user-agent": user_agent.chrome})
  soup = BeautifulSoup(response.content, "html.parser")

  ## Then parse the HTML, extract title and article
  may_title = soup.find(attrs={'class':'td-post-title'}).text.replace('\n','')
  may_title = (re.findall('Think(.*)May', may_title))
  may_text = soup.find(attrs={'class':'td-post-content'}).text.replace('\n','')
  if may_title == []:
    print("Title could not be found")
    continue
  else:
    print(f"Title of the article is : {may_title} \nText of the article is : {may_text}")
    
  # get the domain as the file name
  domain = " ".join(may_title) 
  description = may_text
  filename = response.url.replace("/",'') + '.txt'
  with open("/content/drive/MyDrive/20211030 Test Assignment/output/" + filename, 'w') as f:
      f.write(domain +'\n' + description)

  # appending all the title and text in a list
  domain = " ".join(may_title) 
  description = may_text
  title_all.append(domain)
  text_all.append(description)

Title of the article is : [' How machine learning will affect your business?'] 
Text of the article is : Machine learning techniques may have been used for years, but recently there has been an explosion in their applications. In fact, in a recent Q3 earnings call, Google CEO Sundar Pichai said “Machine learning is a core, transformative way by which we’re re-thinking how we’re doing everything.” And they’re far from the only business making that claim.In the past, successful use of machine learning algorithms required bespoke algorithms and huge R&D budgets, but all that is changing. IBM Watson, Microsoft Azure, Amazon, and Alibaba all launched turnkey cloud-based machine-learning SaaS solutions in 2015. At the same time startups like Idibon, MetaMind, Dato, and MonkeyLearn have built machine learning products that companies can take advantage of.Gartner already puts machine learning at the top of its hype curve, and no: machine learning won’t replace all of your employees with comput

In [ ]:
# adding title and text to the dataframe
may_df.insert(2,'Title', title_all)
may_df.insert(3,'Text', text_all)
may_df

,URL_ID,URL,Title,Text,status,date,year,month
8,45.0,https://insights.blackcoffer.com/how-machine-l...,How machine learning will affect your business?,Machine learning techniques may have been used...,All good,2021-05-29,2021,5
9,46.0,https://insights.blackcoffer.com/deep-learning...,Deep learning impact on areas of e-learning?,eLearning as technology becomes more affordabl...,All good,2021-05-29,2021,5
10,47.0,https://insights.blackcoffer.com/how-to-protec...,How to protect future data and its privacy?,"Before the internet, information was in some w...",All good,2021-05-29,2021,5
11,48.0,https://insights.blackcoffer.com/how-machines-...,"How Machines, AI, Automations, and Robo-human...",We all hear day in and day out that we amidst ...,All good,2021-05-29,2021,5
12,49.0,https://insights.blackcoffer.com/ai-human-robo...,How Robo Human will Impact the Future?,It’s the year 2060. An automaton in a Research...,All good,2021-05-29,2021,5
13,50.0,https://insights.blackcoffer.com/how-ai-will-c...,How AI will change the World?,AbstractThe way work is being done now is dest...,All good,2021-05-29,2021,5
14,51.0,https://insights.blackcoffer.com/future-of-wor...,Future of Work: How AI Has Entered the Workplace,AbstractArtificial intelligence and employment...,All good,2021-05-28,2021,5
15,52.0,https://insights.blackcoffer.com/ai-tool-alexa...,How machine learning used in finance and bank...,Through AI tools like natural language process...,All good,2021-05-27,2021,5
16,53.0,https://insights.blackcoffer.com/ai-healthcare...,How AI will impact the future of work?,AI experts believe it’s going to be one of the...,All good,2021-05-26,2021,5
17,54.0,https://insights.blackcoffer.com/all-you-need-...,All you need to know about online marketing,Ever wondered how you get notified of the pr...,All good,2021-05-05,2021,5


### **June**

In [ ]:
june_df = df.loc[df['month']==6]

In [ ]:
#Function to extract Title and text from the article published in June
## create an array with URLs
urls = june_df['URL']

## initializing the UserAgent object
user_agent = UserAgent()

# defining empty list for title and text
title_all = []
text_all = []

#scrape elements
for url in urls:
  response = requests.get(url, headers={"user-agent": user_agent.chrome})
  soup = BeautifulSoup(response.content, "html.parser")

  ## Then parse the HTML, extract title and article
  june_title = soup.find(attrs={'class':'td-post-title'}).text.replace('\n','')
  june_title = (re.findall('Think(.*)June', june_title))
  june_text = soup.find(attrs={'class':'td-post-content'}).text.replace('\n','')
  if june_title == []:
    print("Title could not be found")
    continue
  else:
    print(f"Title of the article is : {june_title} \nText of the article is : {june_text}")
    
  # get the domain as the file name
  domain = " ".join(june_title) 
  description = june_text
  filename = response.url.replace("/",'') + '.txt'
  with open("/content/drive/MyDrive/20211030 Test Assignment/output/" + filename, 'w') as f:
      f.write(domain +'\n' + description)

  # appending all the title and text in a list
  domain = " ".join(june_title) 
  description = june_text
  title_all.append(domain)
  text_all.append(description)

Title of the article is : [' AI in healthcare to Improve Patient Outcomes'] 
Text of the article is : Introduction“If anything kills over 10 million people in the next few decades, it will be a highly infectious virus rather than a war. Not missiles but microbes.” Bill Gates’s remarks at a TED conference in 2014, right after the world had avoided the Ebola outbreak. When the new, unprecedented, invisible virus hit us, it met an overwhelmed and unprepared healthcare system and oblivious population. This public health emergency demonstrated our lack of scientific consideration and underlined the alarming need for robust innovations in our health and medical facilities. For the past few years, artificial intelligence has proven to be of tangible potential in the healthcare sectors, clinical practices, translational medical and biomedical research.After the first case was detected in China on December 31st 2019, it was an AI program developed by BlueDot that alerted the world about the pan

In [ ]:
# adding title and text to the dataframe
june_df.insert(2,'Title', title_all)
june_df.insert(3,'Text', text_all)
june_df

,URL_ID,URL,Title,Text,status,date,year,month
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction“If anything kills over 10 million...,All good,2021-06-26,2021,6
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ...",All good,2021-06-25,2021,6
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The...,IntroductionAI is rapidly evolving in the empl...,All good,2021-06-25,2021,6
3,40.0,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future ...,“Anything that could give rise to smarter-than...,All good,2021-06-25,2021,6
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...,All good,2021-06-25,2021,6
5,42.0,https://insights.blackcoffer.com/man-and-machi...,Will machine replace the human in the future ...,IntroductionWhere is this disruptive technolog...,All good,2021-06-24,2021,6
6,43.0,https://insights.blackcoffer.com/in-future-or-...,How humans and machines are evolving to work ...,In future or in upcoming years humans and mach...,All good,2021-06-24,2021,6
50,87.0,https://insights.blackcoffer.com/impact-of-cov...,Impact of COVID-19 pandemic on Tourism & Avia...,"As the Coronavirus pandemic unfolds, most indu...",All good,2020-06-22,2020,6
51,88.0,https://insights.blackcoffer.com/impact-of-cov...,Impact of COVID-19 pandemic on sports events ...,The COVID-19 pandemic has caused the most sign...,All good,2020-06-23,2020,6
52,89.0,https://insights.blackcoffer.com/changing-land...,Changing landscape and emerging trends in the...,The COVID 19 pandemic has reshaped the fundame...,All good,2020-06-22,2020,6


### **July**

In [ ]:
july_df = df.loc[df['month']==7]

In [ ]:
#Function to extract Title and text from the article published in July
## create an array with URLs
urls = july_df['URL']

## initializing the UserAgent object
user_agent = UserAgent()

# defining empty list for title and text
title_all = []
text_all = []

#scrape elements
for url in urls:
  response = requests.get(url, headers={"user-agent": user_agent.chrome})
  soup = BeautifulSoup(response.content, "html.parser")

  ## Then parse the HTML, extract title and article
  july_title = soup.find(attrs={'class':'td-post-title'}).text.replace('\n','')
  july_title = (re.findall('Think(.*)July', july_title))
  july_text = soup.find(attrs={'class':'td-post-content'}).text.replace('\n','')
  if july_title == []:
    print("Title could not be found")
    continue
  else:
    print(f"Title of the article is : {july_title} \nText of the article is : {july_text}")

# get the domain as the file name
  domain = " ".join(july_title) 
  description = july_text
  filename = response.url.replace("/",'') + '.txt'
  with open("/content/drive/MyDrive/20211030 Test Assignment/output/" + filename, 'w') as f:
      f.write(domain +'\n' + description)

  # appending all the title and text in a list
  domain = " ".join(july_title) 
  description = july_text
  title_all.append(domain)
  text_all.append(description)

Title of the article is : [' Human Rights Outlook'] 
Text of the article is : Climate change, technological advances, and unprecedented levels of inequality could mean that global disruption becomes a norm in the coming decades.What is changing?Read on to discover what our robot, Athena, has found on the future of this topic and act accordingly for you, your family, and organization in the years ahead. Make notes on issues that could affect you and them as you go.If you are new to foresight, we recommend you view this slide presentation first to get the best out of this reportSpeed read Athena’s high-level take-outs on the left of each slide, or delve deeper into her findings on the right.AnalysisFor a more detailed explanation of the graphics used in this presentation please click here. All outlooks based on the time period 2020-2070 and what’s likely to be happening in 2025 at a 95% confidence level unless otherwise stated. Please contact us for longer-term outlooks.VIDEO SUMMARYTo r

In [ ]:
july_df

,URL_ID,URL,status,date,year,month
54,91.0,https://insights.blackcoffer.com/human-rights-...,All good,2020-07-01,2020,7
55,92.0,https://insights.blackcoffer.com/how-voice-sea...,All good,2020-07-29,2020,7
111,148.0,https://insights.blackcoffer.com/big-data-anal...,All good,2019-07-19,2019,7
112,149.0,https://insights.blackcoffer.com/business-anal...,All good,2019-07-19,2019,7
113,150.0,https://insights.blackcoffer.com/challenges-an...,All good,2019-07-19,2019,7


In [ ]:
# adding title and text to the dataframe
july_df.insert(2,'Title', title_all)
july_df.insert(3,'Text', text_all)
july_df

,URL_ID,URL,Title,Text,status,date,year,month
54,91.0,https://insights.blackcoffer.com/human-rights-...,Human Rights Outlook,"Climate change, technological advances, and un...",All good,2020-07-01,2020,7
55,92.0,https://insights.blackcoffer.com/how-voice-sea...,How Voice search makes your business a succes...,Finding ways to make using the Internet easier...,All good,2020-07-29,2020,7
111,148.0,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare,Quality and affordable healthcare is a vision ...,All good,2019-07-19,2019,7
112,149.0,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry,Analytics is a statistical scientific process ...,All good,2019-07-19,2019,7
113,150.0,https://insights.blackcoffer.com/challenges-an...,Challenges and Opportunities of Big Data in H...,Big DataTo begin with I shall first like to ex...,All good,2019-07-19,2019,7


### **August**

In [ ]:
aug_df = df.loc[df['month']==8]

In [ ]:
#Function to extract Title and text from the article published in August
## create an array with URLs
urls = aug_df['URL']

## initializing the UserAgent object
user_agent = UserAgent()

# defining empty list for title and text
title_all = []
text_all = []

#scrape elements
for url in urls:
  response = requests.get(url, headers={"user-agent": user_agent.chrome})
  soup = BeautifulSoup(response.content, "html.parser")

  ## Then parse the HTML, extract title and article
  aug_title = soup.find(attrs={'class':'td-post-title'}).text.replace('\n','')
  aug_title = (re.findall('Think(.*)August', aug_title))
  aug_text = soup.find(attrs={'class':'td-post-content'}).text.replace('\n','')
  if aug_title == []:
    print("Title could not be found")
    continue
  else:
    print(f"Title of the article is : {aug_title} \nText of the article is : {aug_text}")

# get the domain as the file name
  domain = " ".join(aug_title) 
  description = aug_text
  filename = response.url.replace("/",'') + '.txt'
  with open("/content/drive/MyDrive/20211030 Test Assignment/output/" + filename, 'w') as f:
      f.write(domain +'\n' + description)

  # appending all the title and text in a list
  domain = " ".join(aug_title) 
  description = aug_text
  title_all.append(domain)
  text_all.append(description)

Title of the article is : [' How does artificial intelligence affect the environment'] 
Text of the article is : When we talk about AI, different people have different perspectives about it. There is one group of people, having good knowledge about the real potential of AI, who believe that AI can be a novel solution to many problems that the world is facing today. There is another group of people who are merely threatened by the thought of AI taking over the world.The field of AI took birth when Alan Turing in 1950 had this thought “Can machines think?”. Later in the 1980s, with the adoption of “expert systems” by the companies around the world, the booming of the field of AI was initiated. Initially, it was a matter of awe for everyone to see the results of what AI can achieve. AI’s growth in the past few decades has been exponential and it has transformed the way we live, work and solve challenges. AI has made its impact in a wide variety of areas including healthcare, education, bu

In [ ]:
# adding title and text to the dataframe
aug_df.insert(2,'Title', title_all)
aug_df.insert(3,'Text', text_all)
aug_df

,URL_ID,URL,Title,Text,status,date,year,month
41,78.0,https://insights.blackcoffer.com/how-does-arti...,How does artificial intelligence affect the e...,"When we talk about AI, different people have d...",All good,2020-08-23,2020,8
42,79.0,https://insights.blackcoffer.com/how-to-overco...,How to Overcome Your Fear of Making Mistakes,"No one can reduce mistakes to zero, but you ca...",All good,2020-08-23,2020,8
43,80.0,https://insights.blackcoffer.com/is-perfection...,Is Perfection the Greatest enemy of Productiv...,What’s perfection really? Does every person ex...,All good,2020-08-23,2020,8
44,81.0,https://insights.blackcoffer.com/global-financ...,Global financial crisis 2008 causes/effects a...,Brief The Financial Crisis of 2008 started as ...,All good,2020-08-23,2020,8
45,82.0,https://insights.blackcoffer.com/gender-divers...,Gender diversity and Equality in the tech ind...,Gender diversity is an equal representation of...,All good,2020-08-20,2020,8
46,83.0,https://insights.blackcoffer.com/how-to-overco...,How to overcome your fear of making mistakes?,The Covid-19 crisis and its fallout including ...,All good,2020-08-20,2020,8
47,84.0,https://insights.blackcoffer.com/how-small-bus...,How Small Business can survive the Coronaviru...,The word pandemic may be the most used word fo...,All good,2020-08-18,2020,8
56,93.0,https://insights.blackcoffer.com/how-the-covid...,How the COVID-19 crisis is redefining jobs an...,The pandemic has shaken the world in the way o...,All good,2020-08-15,2020,8
57,94.0,https://insights.blackcoffer.com/how-to-increa...,How to increase social media engagement for m...,Social media is such a staple of our evolving ...,All good,2020-08-15,2020,8
110,147.0,https://insights.blackcoffer.com/the-future-of...,The future of Investing,What Is an Investment?An investment is a resou...,All good,2019-08-31,2019,8


**September**

In [ ]:
sept_df = df.loc[df['month']==9]

In [ ]:
#Function to extract Title and text from the article published in September
## create an array with URLs
urls = sept_df['URL']

## initializing the UserAgent object
user_agent = UserAgent()

# defining empty list for title and text
title_all = []
text_all = []

#scrape elements
for url in urls:
  response = requests.get(url, headers={"user-agent": user_agent.chrome})
  soup = BeautifulSoup(response.content, "html.parser")

  ## Then parse the HTML, extract title and article
  sept_title = soup.find(attrs={'class':'td-post-title'}).text.replace('\n','')
  sept_title = (re.findall('Think(.*)September', sept_title))
  sept_text = soup.find(attrs={'class':'td-post-content'}).text.replace('\n','')
  if sept_title == []:
    print("Title could not be found")
    continue
  else:
    print(f"Title of the article is : {sept_title} \nText of the article is : {sept_text}")

# get the domain as the file name
  domain = " ".join(sept_title) 
  description = sept_text
  filename = response.url.replace("/",'') + '.txt'
  with open("/content/drive/MyDrive/20211030 Test Assignment/output/" + filename, 'w') as f:
      f.write(domain +'\n' + description)

  # appending all the title and text in a list
  domain = " ".join(sept_title) 
  description = sept_text
  title_all.append(domain)
  text_all.append(description)

Title of the article is : [' Why does your business need a chatbot?'] 
Text of the article is : Mr. Sakamoto is a bonsai artist, lives downtown Kyoto, Japan. Bonsai is a Japanese art form that is the cultivation of small trees in a small or medium container. He is in this field of business for the last ten years and he loves what he does on a daily basis. People from different cities in Japan, come to buy his artwork. But, in the early stage of his business, he was just another regular Bosai artist. His popularity has increased because of the stories he tells for every art he is selling. His customer immediately loved him for this while buying A Bonsai art. He loved to sell his customer with his artwork and a story related to that product which resurrects his presence through his art in customers. Soon enough his business thrived and nowadays he is always busy. After the digital revolution, although his sales increased, he was not happy. He was not satisfied. Something is eating up his

In [ ]:
# adding title and text to the dataframe
sept_df.insert(2,'Title', title_all)
sept_df.insert(3,'Text', text_all)
sept_df

,URL_ID,URL,Title,Text,status,date,year,month
38,75.0,https://insights.blackcoffer.com/why-does-your...,Why does your business need a chatbot?,"Mr. Sakamoto is a bonsai artist, lives downtow...",All good,2020-09-22,2020,9
39,76.0,https://insights.blackcoffer.com/how-you-lead-...,How you lead a project or a team without any ...,Have you ever wondered what’s common between S...,All good,2020-09-22,2020,9
40,77.0,https://insights.blackcoffer.com/can-you-be-gr...,Can You Be Great Leader Without Technical Exp...,The word “leadership” can bring to mind a vari...,All good,2020-09-22,2020,9
108,145.0,https://insights.blackcoffer.com/blockchain-in...,Blockchain in Fintech,Disruption in Fintech – The Lightning NetworkB...,All good,2019-09-14,2019,9
109,146.0,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments,Reconciling with the financial realities of an...,All good,2019-09-07,2019,9


**October**

In [ ]:
oct_df = df.loc[df['month']==10]

In [ ]:
#Function to extract Title and text from the article published in October
## create an array with URLs
urls = oct_df['URL']

## initializing the UserAgent object
user_agent = UserAgent()

# defining empty list for title and text
title_all = []
text_all = []

#scrape elements
for url in urls:
  response = requests.get(url, headers={"user-agent": user_agent.chrome})
  soup = BeautifulSoup(response.content, "html.parser")

  ## Then parse the HTML, extract title and article
  oct_title = soup.find(attrs={'class':'td-post-title'}).text.replace('\n','')
  oct_title = (re.findall('Think(.*)October', oct_title))
  oct_text = soup.find(attrs={'class':'td-post-content'}).text.replace('\n','')
  if oct_title == []:
    print("Title could not be found")
    continue
  else:
    print(f"Title of the article is : {oct_title} \nText of the article is : {oct_text}")

# get the domain as the file name
  domain = " ".join(oct_title) 
  description = oct_text
  filename = response.url.replace("/",'') + '.txt'
  with open("/content/drive/MyDrive/20211030 Test Assignment/output/" + filename, 'w') as f:
      f.write(domain +'\n' + description)

  # appending all the title and text in a list
  domain = " ".join(oct_title) 
  description = oct_text
  title_all.append(domain)
  text_all.append(description)

Title of the article is : [' Are we any closer to preventing a nuclear holocaust?'] 
Text of the article is : One thought always comes to my mind…What if we lived without that dander hanging over our heads?And you all know what is that danger is. For more than 70 years the world has faced the very real threat of nuclear war.Nuclear war is a real and growing threat. The United States and Russia have left critical agreements and treaties, while actively planning to add new types of weapons to their arsenals.Meanwhile, US nuclear policy remains rooted in the Cold War, increasing the risk that nuclear weapons could be used again.It doesn’t have to be this way. With the right policy changes and a commitment to diplomacy, the United States can be a leader in reducing the nuclear threat and we can also help them. But the question arises is what are the activities are being done for the prevention of nuclear holocaust?And they are :Pressuring Congress to support change,Holding the White House 

In [ ]:
oct_df

,URL_ID,URL,status,date,year,month
33,70.0,https://insights.blackcoffer.com/are-we-any-cl...,All good,2020-10-23,2020,10
34,71.0,https://insights.blackcoffer.com/will-technolo...,All good,2020-10-22,2020,10
35,72.0,https://insights.blackcoffer.com/will-we-ever-...,All good,2020-10-22,2020,10
36,73.0,https://insights.blackcoffer.com/will-we-ever-...,All good,2020-10-22,2020,10
37,74.0,https://insights.blackcoffer.com/what-is-the-c...,All good,2020-10-22,2020,10


In [ ]:
# adding title and text to the dataframe
oct_df.insert(2,'Title', title_all)
oct_df.insert(3,'Text', text_all)
oct_df

,URL_ID,URL,Title,Text,status,date,year,month
33,70.0,https://insights.blackcoffer.com/are-we-any-cl...,Are we any closer to preventing a nuclear hol...,One thought always comes to my mind…What if we...,All good,2020-10-23,2020,10
34,71.0,https://insights.blackcoffer.com/will-technolo...,Will technology eliminate the need for animal...,Imagine yourself as a being who can speak but ...,All good,2020-10-22,2020,10
35,72.0,https://insights.blackcoffer.com/will-we-ever-...,Will we ever understand the nature of conscio...,IntroductionThe definition of consciousness ha...,All good,2020-10-22,2020,10
36,73.0,https://insights.blackcoffer.com/will-we-ever-...,Will we ever colonize outer space?,"Nature has blessed humanity with earth, a roun...",All good,2020-10-22,2020,10
37,74.0,https://insights.blackcoffer.com/what-is-the-c...,What is the chance Homo sapiens will survive ...,We’ve really done it this year. Like an insati...,All good,2020-10-22,2020,10


**November**

In [ ]:
nov_df = df.loc[df['month']==11]

In [ ]:
#Function to extract Title and text from the article published in November
## create an array with URLs
urls = nov_df['URL']

## initializing the UserAgent object
user_agent = UserAgent()

# defining empty list for title and text
title_all = []
text_all = []

#scrape elements
for url in urls:
  response = requests.get(url, headers={"user-agent": user_agent.chrome})
  soup = BeautifulSoup(response.content, "html.parser")

  ## Then parse the HTML, extract title and article
  nov_title = soup.find(attrs={'class':'td-post-title'}).text.replace('\n','')
  nov_title = (re.findall('Think(.*)November', nov_title))
  nov_text = soup.find(attrs={'class':'td-post-content'}).text.replace('\n','')
  if nov_title == []:
    print("Title could not be found")
    continue
  else:
    print(f"Title of the article is : {nov_title} \nText of the article is : {nov_text}")

# get the domain as the file name
  domain = " ".join(nov_title) 
  description = nov_text
  filename = response.url.replace("/",'') + '.txt'
  with open("/content/drive/MyDrive/20211030 Test Assignment/output/" + filename, 'w') as f:
      f.write(domain +'\n' + description)

  # appending all the title and text in a list
  domain = " ".join(nov_title) 
  description = nov_text
  title_all.append(domain)
  text_all.append(description)

Title of the article is : [' Impacts of COVID 19 on Food products'] 
Text of the article is : Some vendors (fruit and vegetable sellers) began venturing out after a few days without explicit permission and immediately faced police harassment. After a few weeks, the government eased restrictions and essential vendors were being permitted to vend (due in large part to the advocacy of vendor organizations and activist networks). However, the cost of doing business, as well as the risk, has gone up significantly, with vendors not having access to wholesale markets and suppliers and having to spend more on travel costs due to travel restrictions in place in the city. Also, with the lockdown still partially in place, the number of buyers has gone down and so have earnings. Due to the harsh summer heat, perishable fruits and vegetables also have a reduced shelf life so vendors are unable to capitalize on whatever produce they do have.The state has recently announced a stimulus package of INR 

In [ ]:
nov_df

,URL_ID,URL,status,date,year,month
48,85.0,https://insights.blackcoffer.com/impacts-of-co...,All good,2021-11-06,2021,11
49,86.0,https://insights.blackcoffer.com/impacts-of-co...,All good,2021-11-06,2021,11
58,95.0,https://insights.blackcoffer.com/impacts-of-co...,All good,2021-11-06,2021,11
95,132.0,https://insights.blackcoffer.com/marketing-dri...,All good,2019-11-25,2019,11
96,133.0,https://insights.blackcoffer.com/continued-dem...,All good,2019-11-30,2019,11


**We can see that Title of URL in index 95 can't be extracted. Lets find it out now.**

In [ ]:
# Left over articles
urls = 'https://insights.blackcoffer.com/marketing-drives-results-with-a-focus-on-problems/'

## initializing the UserAgent object
user_agent = UserAgent()

#scrape elements
response = requests.get(urls, headers={"user-agent": user_agent.chrome})
soup = BeautifulSoup(response.content, "html.parser")

## Then parse the HTML, extract title and article
nov_title = soup.find(attrs={'class':'td-post-title'}).text.replace('\n','')
nov_title = (re.findall('Marketing(.*)November', nov_title))
nov_text = soup.find(attrs={'class':'td-post-content'}).text.replace('\n','')
nov_text = re.sub(r'http\S+', '', nov_text)

print(f"Title of the article is : {nov_title} \nText of the article is : {nov_text}")

# get the domain as the file name
domain = " ".join(nov_title) 
description = nov_text
filename = response.url.replace("/",'') + '.txt'
with open("/content/drive/MyDrive/20211030 Test Assignment/output/" + filename, 'w') as f:
  f.write(domain +'\n' + description)

# appending all the title and text in a list
domain = " ".join(nov_title) 
description = nov_text
title_all.append(domain)
text_all.append(description)

Title of the article is : [' Marketing Drives Results With A Focus On Problems'] 
Text of the article is : Whenthe British ruled India, many Indiansaccepted to work under their policy, and they did not try to rebel against the British fortheir presence in Indian lands; They recognized the benefits of British rule. Whatwas the factor that made Indians accept theBritish government? The perception aboutBritish-policies (about trading, pay-scale, and positioning in the military) had beendone through various marketing channels in such a way that it brainwashed people of the nation, and many were ready to accept the British. So the Marketing is like fire when smartlyhandled; it can destroy your enemies, but when not it can burn you.  Marketing is to design and develop a pathway from product or service to the customer through a product, price, place and promotion strategies. Marketing depends on a variety of parameters that can be classified broadly, boiling down to vital three. These are ‘Wh

In [ ]:
# rearranging dataframe 
nov_df_re =nov_df.reindex([48,49,58,96,95])
nov_df_re

,URL_ID,URL,status,date,year,month
48,85.0,https://insights.blackcoffer.com/impacts-of-co...,All good,2021-11-06,2021,11
49,86.0,https://insights.blackcoffer.com/impacts-of-co...,All good,2021-11-06,2021,11
58,95.0,https://insights.blackcoffer.com/impacts-of-co...,All good,2021-11-06,2021,11
96,133.0,https://insights.blackcoffer.com/continued-dem...,All good,2019-11-30,2019,11
95,132.0,https://insights.blackcoffer.com/marketing-dri...,All good,2019-11-25,2019,11


In [ ]:
# adding title and text to the dataframe
nov_df_re.insert(2,'Title', title_all)
nov_df_re.insert(3,'Text', text_all)
nov_df_re

,URL_ID,URL,Title,Text,status,date,year,month
48,85.0,https://insights.blackcoffer.com/impacts-of-co...,Impacts of COVID 19 on Food products,Some vendors (fruit and vegetable sellers) beg...,All good,2021-11-06,2021,11
49,86.0,https://insights.blackcoffer.com/impacts-of-co...,Impacts of COVID 19 on Vegetable Vendors,The COVID-19 pandemic has grown into one of th...,All good,2021-11-06,2021,11
58,95.0,https://insights.blackcoffer.com/impacts-of-co...,Impacts of COVID 19 on Streets Sides Food Stalls,Petty Entrepreneurs like Streetside Food Stall...,All good,2021-11-06,2021,11
96,133.0,https://insights.blackcoffer.com/continued-dem...,Continued Demand for Sustainability,The business of business is no longer to do ju...,All good,2019-11-30,2019,11
95,132.0,https://insights.blackcoffer.com/marketing-dri...,Marketing Drives Results With A Focus On Prob...,"Whenthe British ruled India, many Indiansaccep...",All good,2019-11-25,2019,11


**December**

In [ ]:
dec_df = df.loc[df['month']==12]

In [ ]:
#Function to extract Title & Text and giving a file name from the article published in November
## create an array with URLs
urls = dec_df['URL']

## initializing the UserAgent object
user_agent = UserAgent()

# defining empty list for title and text
title_all = []
text_all = []

#scrape elements
for url in urls:
  response = requests.get(url, headers={"user-agent": user_agent.chrome})
  soup = BeautifulSoup(response.content, "html.parser")

  ## Then parse the HTML, extract title and article
  dec_title = soup.find(attrs={'class':'td-post-title'}).text.replace('\n','')
  dec_title = (re.findall('Think(.*)December', dec_title))
  dec_text = soup.find(attrs={'class':'td-post-content'}).text.replace('\n','')
  
  if dec_title == []:
    print("Title could not be found")
  else:
    print(f"Title of the article is : {dec_title} \nText of the article is : {dec_text}")

# get the domain as the file name
  domain = " ".join(dec_title) 
  description = dec_text
  filename = response.url.replace("/",'') + '.txt'
  with open("/content/drive/MyDrive/20211030 Test Assignment/output/" + filename, 'w') as f:
      f.write(domain +'\n' + description)

  # appending all the title and text in a list
  domain = " ".join(dec_title) 
  description = dec_text
  title_all.append(domain)
  text_all.append(description)

Title of the article is : [' Can robots tackle late-life loneliness?'] 
Text of the article is : In this era, everyone is busy in his life. No one spares time for someone. so robots tackle life loneliness? We have no time to stand and stare. This causes loneliness. Loneliness is due to fast-moving life and hurry and worry. In the 21st century, everyone is spending most of their time earning money. They are day-by-day transforming into money-earning machines. Due to his perception, we are losing our relations as we can’t give enough time to our families. They are busy earning their livelihood. They never get satisfies and always want more. Everyone is in a race of earning and then sends his children far away from himself to make them well settled in order to maintain their status. But one faces the consequences of this kind of lifestyle in his old age. People usually don’t spend time with their children when their children are in their childhood but they yearn for the company of their c

In [ ]:
# adding title and text to the dataframe
dec_df.insert(2,'Title', title_all)
dec_df.insert(3,'Text', text_all)
dec_df

,URL_ID,URL,Title,Text,status,date,year,month
30,67.0,https://insights.blackcoffer.com/can-robots-ta...,Can robots tackle late-life loneliness?,"In this era, everyone is busy in his life. No ...",All good,2020-12-02,2020,12
31,68.0,https://insights.blackcoffer.com/embedding-car...,Embedding care robots into society and practi...,"Care Robots, as the name suggests, are robots ...",All good,2020-12-02,2020,12
32,69.0,https://insights.blackcoffer.com/management-ch...,Management challenges for future digitalizati...,Management acts as a guide to a group of peopl...,All good,2020-12-02,2020,12


In [ ]:
# adding all dataframes into a single dataframe
new_df = pd.concat([jan_df, feb_df, mar_df_re, apr_df_re, may_df, june_df, july_df, aug_df, sept_df, oct_df, nov_df_re, dec_df], axis=0, join='inner')

In [ ]:
# adding title and text in one column as full text
new_df['text'] = new_df['Title'] + new_df['Text']

In [ ]:
# dropping Title and Text column
new_df.drop(['Title','Text'], axis=1,inplace=True)

In [ ]:
new_df

,URL_ID,URL,status,date,year,month,text
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,What patients like and dislike about telemedi...
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,How we forecast future technologies?TECHNOLOG...
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,What is the future of mobile apps?From what I...
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,Impact of AI in health and medicineAI allows ...
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,What is the difference between Artificial Int...
...,...,...,...,...,...,...,...
96,133.0,https://insights.blackcoffer.com/continued-dem...,All good,2019-11-30,2019,11,Continued Demand for SustainabilityThe busine...
95,132.0,https://insights.blackcoffer.com/marketing-dri...,All good,2019-11-25,2019,11,Marketing Drives Results With A Focus On Prob...
30,67.0,https://insights.blackcoffer.com/can-robots-ta...,All good,2020-12-02,2020,12,Can robots tackle late-life loneliness?In thi...
31,68.0,https://insights.blackcoffer.com/embedding-car...,All good,2020-12-02,2020,12,Embedding care robots into society and practi...


# **Textual Analysis**

### Text Cleaning

In [ ]:
# Removing everything except the alphabets
def clean_text(x):
    return re.sub(r"[^a-zA-Z ]"," ",str(x))

In [ ]:
# applying above function
new_df['text'] = new_df['text'].apply(clean_text)
new_df

,URL_ID,URL,status,date,year,month,text
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,What patients like and dislike about telemedi...
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,How we forecast future technologies TECHNOLOG...
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,What is the future of mobile apps From what I...
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,Impact of AI in health and medicineAI allows ...
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,What is the difference between Artificial Int...
...,...,...,...,...,...,...,...
96,133.0,https://insights.blackcoffer.com/continued-dem...,All good,2019-11-30,2019,11,Continued Demand for SustainabilityThe busine...
95,132.0,https://insights.blackcoffer.com/marketing-dri...,All good,2019-11-25,2019,11,Marketing Drives Results With A Focus On Prob...
30,67.0,https://insights.blackcoffer.com/can-robots-ta...,All good,2020-12-02,2020,12,Can robots tackle late life loneliness In thi...
31,68.0,https://insights.blackcoffer.com/embedding-car...,All good,2020-12-02,2020,12,Embedding care robots into society and practi...


### Removing Stopwords

In [ ]:
# function to extract stopwords from folder
file_list = glob.glob(os.path.join(os.getcwd(), "/content/drive/MyDrive/20211030 Test Assignment/StopWords", "*.txt"))

corpus = []

for file_path in file_list:
    with open(file_path, encoding='utf8', errors='ignore') as f_input:
      corpus.append(f_input.read())
      string = " ".join( corpus )
      new_stopwords = re.sub(r'http\S+', '', string)
      new_stopwords = new_stopwords.replace('Surnames from 1990 census > .002%.  www.census.gov.genealogy/names/dist.all.last','')
      new_stopwords = re.split(r'[|,\n]', new_stopwords)
      new_stopwords.remove(' ')

print(new_stopwords)

['SMITH  ', 'JOHNSON', 'WILLIAMS', 'JONES', 'BROWN', 'DAVIS', 'MILLER', 'WILSON', 'MOORE', 'TAYLOR', 'ANDERSON', 'THOMAS', 'JACKSON', 'WHITE', 'HARRIS', 'MARTIN', 'THOMPSON', 'GARCIA', 'MARTINEZ', 'ROBINSON', 'CLARK', 'RODRIGUEZ', 'LEWIS', 'LEE', 'WALKER', 'HALL', 'ALLEN', 'YOUNG', 'HERNANDEZ', 'KING', 'WRIGHT', 'LOPEZ', 'HILL', 'SCOTT', 'GREEN', 'ADAMS', 'BAKER', 'GONZALEZ', 'NELSON', 'CARTER', 'MITCHELL', 'PEREZ', 'ROBERTS', 'TURNER', 'PHILLIPS', 'CAMPBELL', 'PARKER', 'EVANS', 'EDWARDS', 'COLLINS', 'STEWART', 'SANCHEZ', 'MORRIS', 'ROGERS', 'REED', 'COOK', 'MORGAN', 'BELL', 'MURPHY', 'BAILEY', 'RIVERA', 'COOPER', 'RICHARDSON', 'COX', 'HOWARD', 'WARD', 'TORRES', 'PETERSON', 'GRAY', 'RAMIREZ', 'JAMES', 'WATSON', 'BROOKS', 'KELLY', 'SANDERS', 'PRICE', 'BENNETT', 'WOOD', 'BARNES', 'ROSS', 'HENDERSON', 'COLEMAN', 'JENKINS', 'PERRY', 'POWELL', 'LONG', 'PATTERSON', 'HUGHES', 'FLORES', 'WASHINGTON', 'BUTLER', 'SIMMONS', 'FOSTER', 'GONZALES', 'BRYANT', 'ALEXANDER', 'RUSSELL', 'GRIFFIN', 'DIAZ'

In [ ]:
# function to remove stopwords
def stopwords(text):
    # removing the stop words 
    text = [word for word in text.split() if word not in new_stopwords]
    # joining the list of words with space separator
    return " ".join(text)

In [ ]:
# removing stopwords from the data
new_df['text'] = new_df['text'].apply(stopwords)
new_df

,URL_ID,URL,status,date,year,month,text
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,What patients dislike telemedicine In today wo...
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,How forecast future technologies TECHNOLOGY pl...
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,What future mobile apps From I learned mobile ...
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,Impact health medicineAI training naturalistic...
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,What difference Artificial Intelligence Machin...
...,...,...,...,...,...,...,...
96,133.0,https://insights.blackcoffer.com/continued-dem...,All good,2019-11-30,2019,11,Continued Demand SustainabilityThe business bu...
95,132.0,https://insights.blackcoffer.com/marketing-dri...,All good,2019-11-25,2019,11,Marketing Drives Results With A Focus On Probl...
30,67.0,https://insights.blackcoffer.com/can-robots-ta...,All good,2020-12-02,2020,12,Can robots tackle late life loneliness In era ...
31,68.0,https://insights.blackcoffer.com/embedding-car...,All good,2020-12-02,2020,12,Embedding care robots society practice Socio t...


### Tokenization

In [ ]:
# function for Tokenization
def tokenizer(txt):  
  tokens = word_tokenize(txt) 
  return tokens

In [ ]:
# Applying Tokenization
new_df['text'] = new_df['text'].apply(tokenizer)
new_df

,URL_ID,URL,status,date,year,month,text
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,"[What, patients, dislike, telemedicine, In, to..."
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,"[How, forecast, future, technologies, TECHNOLO..."
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,"[What, future, mobile, apps, From, I, learned,..."
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,"[Impact, health, medicineAI, training, natural..."
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,"[What, difference, Artificial, Intelligence, M..."
...,...,...,...,...,...,...,...
96,133.0,https://insights.blackcoffer.com/continued-dem...,All good,2019-11-30,2019,11,"[Continued, Demand, SustainabilityThe, busines..."
95,132.0,https://insights.blackcoffer.com/marketing-dri...,All good,2019-11-25,2019,11,"[Marketing, Drives, Results, With, A, Focus, O..."
30,67.0,https://insights.blackcoffer.com/can-robots-ta...,All good,2020-12-02,2020,12,"[Can, robots, tackle, late, life, loneliness, ..."
31,68.0,https://insights.blackcoffer.com/embedding-car...,All good,2020-12-02,2020,12,"[Embedding, care, robots, society, practice, S..."


### Lemmatization

Lemmatization is responsible for grouping different inflected forms of words into the root form, having the same meaning.

In [ ]:
# Applying Lemmatization
def clean_text(text):
    le=WordNetLemmatizer()
    word_tokens=word_tokenize(text)
    tokens=[le.lemmatize(w) for w in word_tokens]
    cleaned_text=" ".join(tokens)
    return cleaned_text

In [ ]:
# Function for Lemmatization
le = WordNetLemmatizer()

def stem_text(text):  
  lemmatize = []
  for word in text:
    lemmatize.append(word)
  return lemmatize

In [ ]:
# Applying Lemmatization
new_df['text'] = new_df['text'].apply(stem_text)
new_df

,URL_ID,URL,status,date,year,month,text
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,"[What, patients, dislike, telemedicine, In, to..."
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,"[How, forecast, future, technologies, TECHNOLO..."
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,"[What, future, mobile, apps, From, I, learned,..."
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,"[Impact, health, medicineAI, training, natural..."
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,"[What, difference, Artificial, Intelligence, M..."
...,...,...,...,...,...,...,...
96,133.0,https://insights.blackcoffer.com/continued-dem...,All good,2019-11-30,2019,11,"[Continued, Demand, SustainabilityThe, busines..."
95,132.0,https://insights.blackcoffer.com/marketing-dri...,All good,2019-11-25,2019,11,"[Marketing, Drives, Results, With, A, Focus, O..."
30,67.0,https://insights.blackcoffer.com/can-robots-ta...,All good,2020-12-02,2020,12,"[Can, robots, tackle, late, life, loneliness, ..."
31,68.0,https://insights.blackcoffer.com/embedding-car...,All good,2020-12-02,2020,12,"[Embedding, care, robots, society, practice, S..."


In [ ]:
#join words fun
def join_words(x):
  return " ".join(set(x))

In [ ]:
# Applying above function
new_df['text'] = new_df['text'].apply(join_words)
new_df

,URL_ID,URL,status,date,year,month,text
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,situations In traveling So doctors facility wo...
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,single beings relationships daily So systemati...
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,credit Build stand easier So Don VR sensors co...
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,effort input kind cancer feedback information ...
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,convey kind defined ability nature So adopted ...
...,...,...,...,...,...,...,...
96,133.0,https://insights.blackcoffer.com/continued-dem...,All good,2019-11-30,2019,11,disclosure dependence Zara minerals accountabi...
95,132.0,https://insights.blackcoffer.com/marketing-dri...,All good,2019-11-25,2019,11,convey Afterimplementing mechanism kind thecus...
30,67.0,https://insights.blackcoffer.com/can-robots-ta...,All good,2020-12-02,2020,12,relations moving Everyone kind heart Singh ach...
31,68.0,https://insights.blackcoffer.com/embedding-car...,All good,2020-12-02,2020,12,observed keeping ability So rehab watch schedu...


## **Creating Dictionary of Positive and Negative words**

**Positive Words**

In [ ]:
# extracting positive words from the folder
file = open("/content/drive/MyDrive/20211030 Test Assignment/MasterDictionary/positive-words.txt", "r")
pos_words = file.read().split()
print(pos_words)

['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation', 'accolade', 'accolades', 'accommodative', 'accomodative', 'accomplish', 'accomplished', 'accomplishment', 'accomplishments', 'accurate', 'accurately', 'achievable', 'achievement', 'achievements', 'achievible', 'acumen', 'adaptable', 'adaptive', 'adequate', 'adjustable', 'admirable', 'admirably', 'admiration', 'admire', 'admirer', 'admiring', 'admiringly', 'adorable', 'adore', 'adored', 'adorer', 'adoring', 'adoringly', 'adroit', 'adroitly', 'adulate', 'adulation', 'adulatory', 'advanced', 'advantage', 'advantageous', 'advantageously', 'advantages', 'adventuresome', 'adventurous', 'advocate', 'advocated', 'advocates', 'affability', 'affable', 'affably', 'affectation', 'affection', 'affectionate', 'affinity', 'affirm', 'affirmation', 'affirmative', 'affluence', 'affluent', 'afford', 'affordable', 'affordably', 'afordable', 'agile', 'agilely', 'agility', 'agreeable', 'ag

**Negative Words**

In [ ]:
file = open("/content/drive/MyDrive/20211030 Test Assignment/MasterDictionary/negative-words.txt", encoding='utf8', errors='ignore')
neg_words = file.read().split()
print(neg_words)

['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted', 'aborts', 'abrade', 'abrasive', 'abrupt', 'abruptly', 'abscond', 'absence', 'absent-minded', 'absentee', 'absurd', 'absurdity', 'absurdly', 'absurdness', 'abuse', 'abused', 'abuses', 'abusive', 'abysmal', 'abysmally', 'abyss', 'accidental', 'accost', 'accursed', 'accusation', 'accusations', 'accuse', 'accuses', 'accusing', 'accusingly', 'acerbate', 'acerbic', 'acerbically', 'ache', 'ached', 'aches', 'achey', 'aching', 'acrid', 'acridly', 'acridness', 'acrimonious', 'acrimoniously', 'acrimony', 'adamant', 'adamantly', 'addict', 'addicted', 'addicting', 'addicts', 'admonish', 'admonisher', 'admonishingly', 'admonishment', 'admonition', 'adulterate', 'adulterated', 'adulteration', 'adulterier', 'adversarial', 'adversary', 'adverse', 'adversity', 'afflict', 'affliction', 'afflictive', 'affront', 'afraid', 'aggravate', 'aggravating', 'aggravation', 'aggression', 'aggress

## **Extracting Derived variables**

**POSITIVE SCORE**

In [ ]:
# Finding positive scores of text
num_pos = new_df['text'].map(lambda x: len([i for i in x.split() if i in pos_words]))
new_df['pos_score'] = num_pos
new_df

,URL_ID,URL,status,date,year,month,text,pos_score
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,situations In traveling So doctors facility wo...,13
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,single beings relationships daily So systemati...,5
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,credit Build stand easier So Don VR sensors co...,19
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,effort input kind cancer feedback information ...,18
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,convey kind defined ability nature So adopted ...,21
...,...,...,...,...,...,...,...,...
96,133.0,https://insights.blackcoffer.com/continued-dem...,All good,2019-11-30,2019,11,disclosure dependence Zara minerals accountabi...,27
95,132.0,https://insights.blackcoffer.com/marketing-dri...,All good,2019-11-25,2019,11,convey Afterimplementing mechanism kind thecus...,15
30,67.0,https://insights.blackcoffer.com/can-robots-ta...,All good,2020-12-02,2020,12,relations moving Everyone kind heart Singh ach...,19
31,68.0,https://insights.blackcoffer.com/embedding-car...,All good,2020-12-02,2020,12,observed keeping ability So rehab watch schedu...,8


**NEGATIVE SCORE**

In [ ]:
# Finding negative scores of text
num_neg = new_df['text'].map(lambda x: len([i for i in x.split() if i in neg_words]))
new_df['neg_score'] = num_neg
new_df

,URL_ID,URL,status,date,year,month,text,pos_score,neg_score
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,situations In traveling So doctors facility wo...,13,9
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,single beings relationships daily So systemati...,5,0
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,credit Build stand easier So Don VR sensors co...,19,9
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,effort input kind cancer feedback information ...,18,7
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,convey kind defined ability nature So adopted ...,21,13
...,...,...,...,...,...,...,...,...,...
96,133.0,https://insights.blackcoffer.com/continued-dem...,All good,2019-11-30,2019,11,disclosure dependence Zara minerals accountabi...,27,24
95,132.0,https://insights.blackcoffer.com/marketing-dri...,All good,2019-11-25,2019,11,convey Afterimplementing mechanism kind thecus...,15,15
30,67.0,https://insights.blackcoffer.com/can-robots-ta...,All good,2020-12-02,2020,12,relations moving Everyone kind heart Singh ach...,19,23
31,68.0,https://insights.blackcoffer.com/embedding-car...,All good,2020-12-02,2020,12,observed keeping ability So rehab watch schedu...,8,2


**POLARITY SCORE**

In [ ]:
# Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
# Range is from -1 to +1
new_df['polarity_score'] = (new_df['pos_score']-new_df['neg_score'])/((new_df['pos_score']+new_df['neg_score'])+0.000001)
new_df

,URL_ID,URL,status,date,year,month,text,pos_score,neg_score,polarity_score
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,situations In traveling So doctors facility wo...,13,9,0.181818
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,single beings relationships daily So systemati...,5,0,1.000000
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,credit Build stand easier So Don VR sensors co...,19,9,0.357143
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,effort input kind cancer feedback information ...,18,7,0.440000
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,convey kind defined ability nature So adopted ...,21,13,0.235294
...,...,...,...,...,...,...,...,...,...,...
96,133.0,https://insights.blackcoffer.com/continued-dem...,All good,2019-11-30,2019,11,disclosure dependence Zara minerals accountabi...,27,24,0.058824
95,132.0,https://insights.blackcoffer.com/marketing-dri...,All good,2019-11-25,2019,11,convey Afterimplementing mechanism kind thecus...,15,15,0.000000
30,67.0,https://insights.blackcoffer.com/can-robots-ta...,All good,2020-12-02,2020,12,relations moving Everyone kind heart Singh ach...,19,23,-0.095238
31,68.0,https://insights.blackcoffer.com/embedding-car...,All good,2020-12-02,2020,12,observed keeping ability So rehab watch schedu...,8,2,0.600000


**SUBJECTIVITY SCORE**

In [ ]:
# Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)
# Range is from 0 to +1
word_count = new_df['text'].apply(lambda x: len((x).split()))
new_df['subj_score'] = (new_df['pos_score']+new_df['neg_score'])/((word_count)+ 0.000001)
new_df

,URL_ID,URL,status,date,year,month,text,pos_score,neg_score,polarity_score,subj_score
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,situations In traveling So doctors facility wo...,13,9,0.181818,0.171875
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,single beings relationships daily So systemati...,5,0,1.000000,0.062500
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,credit Build stand easier So Don VR sensors co...,19,9,0.357143,0.067633
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,effort input kind cancer feedback information ...,18,7,0.440000,0.090909
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,convey kind defined ability nature So adopted ...,21,13,0.235294,0.085642
...,...,...,...,...,...,...,...,...,...,...,...
96,133.0,https://insights.blackcoffer.com/continued-dem...,All good,2019-11-30,2019,11,disclosure dependence Zara minerals accountabi...,27,24,0.058824,0.114350
95,132.0,https://insights.blackcoffer.com/marketing-dri...,All good,2019-11-25,2019,11,convey Afterimplementing mechanism kind thecus...,15,15,0.000000,0.064378
30,67.0,https://insights.blackcoffer.com/can-robots-ta...,All good,2020-12-02,2020,12,relations moving Everyone kind heart Singh ach...,19,23,-0.095238,0.167331
31,68.0,https://insights.blackcoffer.com/embedding-car...,All good,2020-12-02,2020,12,observed keeping ability So rehab watch schedu...,8,2,0.600000,0.117647


## **Analysis of Readability**

**AVERAGE SENTENCE LENGTH**

In [ ]:
# Average Sentence Length = the number of words / the number of sentences
num_sentences = word_count.count()
new_df['avg_sent_len'] = word_count/num_sentences
new_df

,URL_ID,URL,status,date,year,month,text,pos_score,neg_score,polarity_score,subj_score,avg_sent_len
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,situations In traveling So doctors facility wo...,13,9,0.181818,0.171875,1.153153
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,single beings relationships daily So systemati...,5,0,1.000000,0.062500,0.720721
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,credit Build stand easier So Don VR sensors co...,19,9,0.357143,0.067633,3.729730
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,effort input kind cancer feedback information ...,18,7,0.440000,0.090909,2.477477
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,convey kind defined ability nature So adopted ...,21,13,0.235294,0.085642,3.576577
...,...,...,...,...,...,...,...,...,...,...,...,...
96,133.0,https://insights.blackcoffer.com/continued-dem...,All good,2019-11-30,2019,11,disclosure dependence Zara minerals accountabi...,27,24,0.058824,0.114350,4.018018
95,132.0,https://insights.blackcoffer.com/marketing-dri...,All good,2019-11-25,2019,11,convey Afterimplementing mechanism kind thecus...,15,15,0.000000,0.064378,4.198198
30,67.0,https://insights.blackcoffer.com/can-robots-ta...,All good,2020-12-02,2020,12,relations moving Everyone kind heart Singh ach...,19,23,-0.095238,0.167331,2.261261
31,68.0,https://insights.blackcoffer.com/embedding-car...,All good,2020-12-02,2020,12,observed keeping ability So rehab watch schedu...,8,2,0.600000,0.117647,0.765766


**PERCENTAGE OF COMPLEX WORDS**

In [ ]:
# syllable per word = no of vowels present in each word.
vowel_per_word = new_df['text'].str.count(r'[aeiou]')

In [ ]:
# Complex words are words in the text that contain more than two syllables.
def vowelwordcount(text):

  # initializing count variable
  count = 0
  word_count = 0

  # creating a list of vowels
  vowels = ["a", "e", "i", "o", "u"]
  for row in text:
      for i in row:
         if i in vowels:
          count += 1
      if count>2:
        word_count += 1
  return word_count

In [ ]:
complex_words = new_df['text'].apply(vowelwordcount)

In [ ]:
# Percentage of Complex words = the number of complex words / the number of words 
# complex words = more than two vowels present in each word.
new_df['percent_complex_words'] = complex_words/word_count
new_df

,URL_ID,URL,status,date,year,month,text,pos_score,neg_score,polarity_score,subj_score,avg_sent_len,percent_complex_words
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,situations In traveling So doctors facility wo...,13,9,0.181818,0.171875,1.153153,7.976562
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,single beings relationships daily So systemati...,5,0,1.000000,0.062500,0.720721,8.000000
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,credit Build stand easier So Don VR sensors co...,19,9,0.357143,0.067633,3.729730,7.398551
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,effort input kind cancer feedback information ...,18,7,0.440000,0.090909,2.477477,8.418182
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,convey kind defined ability nature So adopted ...,21,13,0.235294,0.085642,3.576577,8.196474
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,133.0,https://insights.blackcoffer.com/continued-dem...,All good,2019-11-30,2019,11,disclosure dependence Zara minerals accountabi...,27,24,0.058824,0.114350,4.018018,8.957399
95,132.0,https://insights.blackcoffer.com/marketing-dri...,All good,2019-11-25,2019,11,convey Afterimplementing mechanism kind thecus...,15,15,0.000000,0.064378,4.198198,8.261803
30,67.0,https://insights.blackcoffer.com/can-robots-ta...,All good,2020-12-02,2020,12,relations moving Everyone kind heart Singh ach...,19,23,-0.095238,0.167331,2.261261,7.486056
31,68.0,https://insights.blackcoffer.com/embedding-car...,All good,2020-12-02,2020,12,observed keeping ability So rehab watch schedu...,8,2,0.600000,0.117647,0.765766,7.447059


**FOG INDEX**

In [ ]:
# Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)
new_df['fog_index'] = 0.4* (new_df['avg_sent_len'] + new_df['percent_complex_words'])
new_df.head()

,URL_ID,URL,status,date,year,month,text,pos_score,neg_score,polarity_score,subj_score,avg_sent_len,percent_complex_words,fog_index
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,situations In traveling So doctors facility wo...,13,9,0.181818,0.171875,1.153153,7.976562,3.651886
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,single beings relationships daily So systemati...,5,0,1.000000,0.062500,0.720721,8.000000,3.488288
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,credit Build stand easier So Don VR sensors co...,19,9,0.357143,0.067633,3.729730,7.398551,4.451312
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,effort input kind cancer feedback information ...,18,7,0.440000,0.090909,2.477477,8.418182,4.358264
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,convey kind defined ability nature So adopted ...,21,13,0.235294,0.085642,3.576577,8.196474,4.709220


**AVG NUMBER OF WORDS PER SENTENCE**

In [ ]:
# total number of words
total_words = word_count.sum()
total_words

43179

In [ ]:
# Average Number of Words Per Sentence = the total number of words / the total number of sentences
new_df['avg_words_per_sen'] = total_words/num_sentences
new_df.head()

,URL_ID,URL,status,date,year,month,text,pos_score,neg_score,polarity_score,subj_score,avg_sent_len,percent_complex_words,fog_index,avg_words_per_sen
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,situations In traveling So doctors facility wo...,13,9,0.181818,0.171875,1.153153,7.976562,3.651886,389.0
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,single beings relationships daily So systemati...,5,0,1.000000,0.062500,0.720721,8.000000,3.488288,389.0
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,credit Build stand easier So Don VR sensors co...,19,9,0.357143,0.067633,3.729730,7.398551,4.451312,389.0
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,effort input kind cancer feedback information ...,18,7,0.440000,0.090909,2.477477,8.418182,4.358264,389.0
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,convey kind defined ability nature So adopted ...,21,13,0.235294,0.085642,3.576577,8.196474,4.709220,389.0


**COMPLEX WORD COUNT**

In [ ]:
new_df['complex_words'] = complex_words
new_df.head()

,URL_ID,URL,status,date,year,month,text,pos_score,neg_score,polarity_score,subj_score,avg_sent_len,percent_complex_words,fog_index,avg_words_per_sen,complex_words
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,situations In traveling So doctors facility wo...,13,9,0.181818,0.171875,1.153153,7.976562,3.651886,389.0,1021
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,single beings relationships daily So systemati...,5,0,1.000000,0.062500,0.720721,8.000000,3.488288,389.0,640
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,credit Build stand easier So Don VR sensors co...,19,9,0.357143,0.067633,3.729730,7.398551,4.451312,389.0,3063
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,effort input kind cancer feedback information ...,18,7,0.440000,0.090909,2.477477,8.418182,4.358264,389.0,2315
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,convey kind defined ability nature So adopted ...,21,13,0.235294,0.085642,3.576577,8.196474,4.709220,389.0,3254


**WORD COUNT**

In [ ]:
new_df['word_count'] = word_count
new_df.head()

,URL_ID,URL,status,date,year,month,text,pos_score,neg_score,polarity_score,subj_score,avg_sent_len,percent_complex_words,fog_index,avg_words_per_sen,complex_words,word_count
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,situations In traveling So doctors facility wo...,13,9,0.181818,0.171875,1.153153,7.976562,3.651886,389.0,1021,128
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,single beings relationships daily So systemati...,5,0,1.000000,0.062500,0.720721,8.000000,3.488288,389.0,640,80
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,credit Build stand easier So Don VR sensors co...,19,9,0.357143,0.067633,3.729730,7.398551,4.451312,389.0,3063,414
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,effort input kind cancer feedback information ...,18,7,0.440000,0.090909,2.477477,8.418182,4.358264,389.0,2315,275
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,convey kind defined ability nature So adopted ...,21,13,0.235294,0.085642,3.576577,8.196474,4.709220,389.0,3254,397


**SYLLABLE PER WORD**

In [ ]:
new_df['syllable_per_word'] = vowel_per_word
new_df.head()

,URL_ID,URL,status,date,year,month,text,pos_score,neg_score,polarity_score,subj_score,avg_sent_len,percent_complex_words,fog_index,avg_words_per_sen,complex_words,word_count,syllable_per_word
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,situations In traveling So doctors facility wo...,13,9,0.181818,0.171875,1.153153,7.976562,3.651886,389.0,1021,128,334
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,single beings relationships daily So systemati...,5,0,1.000000,0.062500,0.720721,8.000000,3.488288,389.0,640,80,194
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,credit Build stand easier So Don VR sensors co...,19,9,0.357143,0.067633,3.729730,7.398551,4.451312,389.0,3063,414,957
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,effort input kind cancer feedback information ...,18,7,0.440000,0.090909,2.477477,8.418182,4.358264,389.0,2315,275,766
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,convey kind defined ability nature So adopted ...,21,13,0.235294,0.085642,3.576577,8.196474,4.709220,389.0,3254,397,1064


**PERSONAL PRONOUNS**

In [ ]:
# Personal Pronouns = find the counts of the words - “I,” “we,” “my,” “ours,” and “us” in the text

In [ ]:
# function to find personal pronouns in the text
def pronounwordcount(text):

  # initializing count variable
  count = 0

  # creating a list of vowels
  pronoun = ['I', 'we', 'my', 'ours', 'us']
  for row in text:
      for i in row:
         if i in pronoun:
          count += 1
  return count

In [ ]:
new_df['personal_pronouns'] = new_df['text'].apply(pronounwordcount)
new_df.head()

,URL_ID,URL,status,date,year,month,text,pos_score,neg_score,polarity_score,subj_score,avg_sent_len,percent_complex_words,fog_index,avg_words_per_sen,complex_words,word_count,syllable_per_word,personal_pronouns
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,situations In traveling So doctors facility wo...,13,9,0.181818,0.171875,1.153153,7.976562,3.651886,389.0,1021,128,334,3
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,single beings relationships daily So systemati...,5,0,1.000000,0.062500,0.720721,8.000000,3.488288,389.0,640,80,194,4
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,credit Build stand easier So Don VR sensors co...,19,9,0.357143,0.067633,3.729730,7.398551,4.451312,389.0,3063,414,957,15
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,effort input kind cancer feedback information ...,18,7,0.440000,0.090909,2.477477,8.418182,4.358264,389.0,2315,275,766,7
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,convey kind defined ability nature So adopted ...,21,13,0.235294,0.085642,3.576577,8.196474,4.709220,389.0,3254,397,1064,9


**AVG WORD LENGTH**

In [ ]:
# Average Word Length = Sum of the total number of characters in each word/Total number of words
# function to find count of characters in each word in the text
def wordcount(text):

  # initializing count variable
  count = 0

  for row in text:
      for i in row:
        count += 1

  return count

In [ ]:
char_in_words = new_df['text'].apply(wordcount)

In [ ]:
new_df['avg_word_len'] = char_in_words/num_sentences
new_df.head()

,URL_ID,URL,status,date,year,month,text,pos_score,neg_score,polarity_score,subj_score,avg_sent_len,percent_complex_words,fog_index,avg_words_per_sen,complex_words,word_count,syllable_per_word,personal_pronouns,avg_word_len
28,65.0,https://insights.blackcoffer.com/telemedicine-...,All good,2021-01-13,2021,1,situations In traveling So doctors facility wo...,13,9,0.181818,0.171875,1.153153,7.976562,3.651886,389.0,1021,128,334,3,9.234234
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,All good,2021-01-13,2021,1,single beings relationships daily So systemati...,5,0,1.000000,0.062500,0.720721,8.000000,3.488288,389.0,640,80,194,4,5.837838
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,All good,2021-02-12,2021,2,credit Build stand easier So Don VR sensors co...,19,9,0.357143,0.067633,3.729730,7.398551,4.451312,389.0,3063,414,957,15,27.666667
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,All good,2021-02-11,2021,2,effort input kind cancer feedback information ...,18,7,0.440000,0.090909,2.477477,8.418182,4.358264,389.0,2315,275,766,7,20.918919
23,60.0,https://insights.blackcoffer.com/difference-be...,All good,2021-03-09,2021,3,convey kind defined ability nature So adopted ...,21,13,0.235294,0.085642,3.576577,8.196474,4.709220,389.0,3254,397,1064,9,29.387387


### Customizing dataframe according to Excel File

In [ ]:
new_df.drop(['status','date','year','month','text',], axis=1, inplace = True)
new_df.rename(columns = {'pos_score' : 'POSITIVE SCORE', 'neg_score': 'NEGATIVE SCORE',
                         'polarity_score' : 'POLARITY SCORE', 'subj_score' : 'SUBJECTIVITY SCORE',
                         'avg_sent_len' : 'AVG SENTENCE LENGTH', 'percent_complex_words' : 'PERCENTAGE OF COMPLEX WORDS',
                         'fog_index' : 'FOG INDEX','avg_words_per_sen' : 'AVG NUMBER OF WORDS PER SENTENCE',
                         'complex_words' : 'COMPLEX WORD COUNT', 'word_count' : 'WORD COUNT', 'syllable_per_word' : 'SYLLABLE PER WORD',
                         'personal_pronouns' : 'PERSONAL PRONOUNS', 'avg_word_len' : 'AVG WORD LENGTH'}, inplace = True)

In [ ]:
new_df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
28,65.0,https://insights.blackcoffer.com/telemedicine-...,13,9,0.181818,0.171875,1.153153,7.976562,3.651886,389.0,1021,128,334,3,9.234234
29,66.0,https://insights.blackcoffer.com/how-we-foreca...,5,0,1.000000,0.062500,0.720721,8.000000,3.488288,389.0,640,80,194,4,5.837838
26,63.0,https://insights.blackcoffer.com/what-is-the-f...,19,9,0.357143,0.067633,3.729730,7.398551,4.451312,389.0,3063,414,957,15,27.666667
27,64.0,https://insights.blackcoffer.com/impact-of-ai-...,18,7,0.440000,0.090909,2.477477,8.418182,4.358264,389.0,2315,275,766,7,20.918919
23,60.0,https://insights.blackcoffer.com/difference-be...,21,13,0.235294,0.085642,3.576577,8.196474,4.709220,389.0,3254,397,1064,9,29.387387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,133.0,https://insights.blackcoffer.com/continued-dem...,27,24,0.058824,0.114350,4.018018,8.957399,5.190167,389.0,3995,446,1364,10,36.054054
95,132.0,https://insights.blackcoffer.com/marketing-dri...,15,15,0.000000,0.064378,4.198198,8.261803,4.984000,389.0,3850,466,1263,14,34.774775
30,67.0,https://insights.blackcoffer.com/can-robots-ta...,19,23,-0.095238,0.167331,2.261261,7.486056,3.898927,389.0,1879,251,603,6,16.972973
31,68.0,https://insights.blackcoffer.com/embedding-car...,8,2,0.600000,0.117647,0.765766,7.447059,3.285130,389.0,633,85,203,4,5.756757


In [ ]:
new_df.sort_values(by=['URL_ID'], axis=0, inplace=True)

In [ ]:
new_df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,44,27,0.239437,0.100141,6.387387,8.544429,5.972726,389.0,6058,709,2015,9,54.648649
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,39,28,0.164179,0.153318,3.936937,7.821510,4.703379,389.0,3418,437,1133,6,30.837838
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,48,24,0.333333,0.120401,5.387387,8.484950,5.548935,389.0,5074,598,1707,10,45.792793
3,40.0,https://insights.blackcoffer.com/will-machine-...,35,19,0.296296,0.112500,4.324324,7.937500,4.904730,389.0,3810,480,1212,15,34.369369
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,38,22,0.266667,0.098847,5.468468,8.085667,5.421654,389.0,4908,607,1593,15,44.288288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146.0,https://insights.blackcoffer.com/blockchain-fo...,20,27,-0.148936,0.116337,3.639640,8.126238,4.706351,389.0,3283,404,1040,8,29.630631
110,147.0,https://insights.blackcoffer.com/the-future-of...,22,11,0.333333,0.061567,4.828829,7.744403,5.029293,389.0,4151,536,1320,26,37.468468
111,148.0,https://insights.blackcoffer.com/big-data-anal...,18,27,-0.200000,0.120643,3.360360,8.077748,4.575243,389.0,3013,373,983,10,27.198198
112,149.0,https://insights.blackcoffer.com/business-anal...,19,7,0.461538,0.087838,2.666667,8.989865,4.662613,389.0,2661,296,875,8,24.063063


### **Writing dataframe to Excel File**

In [ ]:
new_df.to_excel('/content/drive/MyDrive/20211030 Test Assignment/Output Data Structure.xlsx', sheet_name='Sheet1',index=False)